In [2]:
import csv
import random
from unidecode import unidecode
import warnings

import pandas as pd
import numpy as np

import hu_core_ud_lg

import nltk
from nltk.corpus import stopwords

from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from elasticsearch.helpers import bulk
from elasticsearch.exceptions import ElasticsearchWarning

from tqdm.notebook import tqdm

In [3]:
es = Elasticsearch("http://localhost:9200")

In [4]:
faq_csv = ".data/faq_with_splits_tokenized.csv"

In [5]:
df = pd.read_csv(faq_csv)

In [6]:
df.head()

,short_question,long_question,answer,main_category,sub_category,split
0,Vállsérülés gyógytorna,Vállsérülés gyógytorna Az lenne a kérdésem hog...,"edzeni kénealapjába véve mindent váll, hát, tr...",Egészség,"Sérülések, balesetek",train
1,Heti hányszor dehidtratáljam a szervezetemet c...,Heti hányszor dehidtratáljam a szervezetemet c...,"Nem értem miért akarod magad dehidratálni, nem...",Egészség,Bio életmód,train
2,A Doctor Who ban ki a vörös hajú lány,A Doctor Who ban ki a vörös hajú lány,Vagy Amy Pond vagy Donna Noble,Szórakozás,"Filmek, sorozatok",train
3,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Egy törpenyuszihoz nagyon sok türelem kell. Pr...,Állatok,Kisemlősök,train
4,Fityma szűkületem van,Fityma szűkületem van 15 éves fiú vagyok és mi...,Valószínű. Esetleg még a fitymaféked lehet röv...,Egészség,Férfiak egészsége,train


In [7]:
df = df[['short_question', 'long_question', 'answer', 'main_category']]

In [12]:
df.head()

,short_question,long_question,answer,_index
0,Vállsérülés gyógytorna,Vállsérülés gyógytorna Az lenne a kérdésem hog...,"edzeni kénealapjába véve mindent váll, hát, tr...",Egészség
1,Heti hányszor dehidtratáljam a szervezetemet c...,Heti hányszor dehidtratáljam a szervezetemet c...,"Nem értem miért akarod magad dehidratálni, nem...",Egészség
2,A Doctor Who ban ki a vörös hajú lány,A Doctor Who ban ki a vörös hajú lány,Vagy Amy Pond vagy Donna Noble,Szórakozás
3,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Egy törpenyuszihoz nagyon sok türelem kell. Pr...,Állatok
4,Fityma szűkületem van,Fityma szűkületem van 15 éves fiú vagyok és mi...,Valószínű. Esetleg még a fitymaféked lehet röv...,Egészség


In [13]:
df.rename(columns = {'main_category': '_index'}, inplace=True)

In [17]:
df['_index'] = df['_index'].apply(lambda x : "questions_" + unidecode(x.lower()))

In [18]:
df.head()

,short_question,long_question,answer,_index
0,Vállsérülés gyógytorna,Vállsérülés gyógytorna Az lenne a kérdésem hog...,"edzeni kénealapjába véve mindent váll, hát, tr...",questions_egeszseg
1,Heti hányszor dehidtratáljam a szervezetemet c...,Heti hányszor dehidtratáljam a szervezetemet c...,"Nem értem miért akarod magad dehidratálni, nem...",questions_egeszseg
2,A Doctor Who ban ki a vörös hajú lány,A Doctor Who ban ki a vörös hajú lány,Vagy Amy Pond vagy Donna Noble,questions_szorakozas
3,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Egy törpenyuszihoz nagyon sok türelem kell. Pr...,questions_allatok
4,Fityma szűkületem van,Fityma szűkületem van 15 éves fiú vagyok és mi...,Valószínű. Esetleg még a fitymaféked lehet röv...,questions_egeszseg


In [19]:
warnings.simplefilter('ignore', ElasticsearchWarning)

In [20]:
def delete_indices():
    indices = es.indices.get('questions*')

    for index in indices:
        print(f"Deleting index '{index}'")
        es.indices.delete(index=index, ignore=[404])

In [21]:
delete_indices()

Deleting index 'questions_allatok'
Deleting index 'questions_egeszseg'
Deleting index 'questions_szamitastechnika'
Deleting index 'questions_szorakozas'


In [24]:
def generate_data():
    
    for index, row in df.iterrows():
        yield {
            "_index": row['_index'],
            "short_question": row.short_question,
            "long_question": row.long_question,
            "answer": row.answer
        }

In [25]:
bulk(es, generate_data())

(153556, [])

In [26]:
def generate_query(question):
    return {
        "query": {
            "match": {
                "long_question": {
                    "query": question,
                }
            }
        }
    }
    
def find_answer(question, index="questions*"):
    body = generate_query(question)
    res = es.search(index=index, body=body)
    
    return res['hits']['hits'][0]['_source']

In [27]:
question = input()

result = find_answer(question)

print(f"Short question:\n{result['short_question']}")
print()
print(f"Long question:\n{result['long_question']}")
print()
print(f"Answer:\n{result['answer']}")

Miért fáj a lábam futás közben?
Short question:
Miért fáj a lábfejem futás közben

Long question:
Miért fáj a lábfejem futás közben Sziasztok Miért fáj a lábfejem és a talp alsó része futás közben Régen amikor nyáron rendszeresen fociztam akkor nem fájt De most elkezdődött az iskola és testnevelés órán sokat futunk Sétáláskor nem fáj csak ha futáskor és a futás után Miért fáj Miért van ez Lúdtalpam nincs Várom a válaszokat Előre is köszönöm a segítséget

Answer:
Nekem volt a lábfejemben szalaghúzódásom, de az álláskor is fájt, bár ha ültem, akkor bármit tehettem vele, nem éreztem semmit.


-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------

In [28]:
index_file = "index.json"
batch_size = 1000

In [29]:
df.head()

,short_question,long_question,answer,_index
0,Vállsérülés gyógytorna,Vállsérülés gyógytorna Az lenne a kérdésem hog...,"edzeni kénealapjába véve mindent váll, hát, tr...",questions_egeszseg
1,Heti hányszor dehidtratáljam a szervezetemet c...,Heti hányszor dehidtratáljam a szervezetemet c...,"Nem értem miért akarod magad dehidratálni, nem...",questions_egeszseg
2,A Doctor Who ban ki a vörös hajú lány,A Doctor Who ban ki a vörös hajú lány,Vagy Amy Pond vagy Donna Noble,questions_szorakozas
3,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Egy törpenyuszihoz nagyon sok türelem kell. Pr...,questions_allatok
4,Fityma szűkületem van,Fityma szűkületem van 15 éves fiú vagyok és mi...,Valószínű. Esetleg még a fitymaféked lehet röv...,questions_egeszseg


In [32]:
nlp = hu_core_ud_lg.load()

In [33]:
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('hungarian'))

list(stop_words)[:5]

['lenni', 'nagyon', 'kell', 'össze', 'pedig']

In [34]:
def filter_stopwords(text):
    return " ". join([w for w in text if not w in stop_words])

In [35]:
def embed(sentence):
    docs = nlp(sentence)  
    avg = sum([doc.vector for doc in docs]) / len(docs)
    return avg

In [36]:
def embed_docs(docs):
    return [embed(filter_stopwords(doc)) for doc in docs]

In [39]:
index_names = df['_index'].unique().tolist()
index_names

['questions_egeszseg',
 'questions_szorakozas',
 'questions_allatok',
 'questions_szamitastechnika']

In [52]:
delete_indices()

Deleting index 'questions_allatok'
Deleting index 'questions_egeszseg'
Deleting index 'questions_szamitastechnika'
Deleting index 'questions_szorakozas'


In [53]:
with open(index_file) as index:
    source = index.read().strip()
    
    for index_name in index_names:
        print(f"Creating index '{index_name}'")
        es.indices.create(index=index_name, body=source)

Creating index 'questions_egeszseg'
Creating index 'questions_szorakozas'
Creating index 'questions_allatok'
Creating index 'questions_szamitastechnika'


In [54]:
def index_batch(docs):
    short_questions = [doc["short_question"] for doc in docs]
    short_questions_vectors = embed_docs(short_questions)

    requests = []
    
    for i, doc, in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["short_question_vector"] = short_questions_vectors[i]
        requests.append(request)
    bulk(es, requests)

In [55]:
def index_data(df):
    num_of_batches = df.shape[0] // batch_size
    
    if df.shape[0] % batch_size != 0:
        num_of_batches += 1

    progress_bar = tqdm(desc=f"Indexing batches", total=num_of_batches, leave=True)
        
    for i in range(num_of_batches):
        batch = df[i * batch_size : (i + 1) * batch_size]
        batch_as_dict = batch.apply(lambda row: row.to_dict(), axis = 1).tolist()

        index_batch(batch_as_dict)
        
        progress_bar.update()
        progress_bar.refresh()

In [56]:
index_data(df)

Indexing batches:   0%|          | 0/154 [00:00<?, ?it/s]

In [57]:
script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, 'short_question_vector') + 1.0",
            "params": {"query_vector": embed(filter_stopwords(question))}
        }
    }
}


response = es.search(
    index='questions*',
    body={
        "size": 10,
        "query": script_query
    }
)

In [58]:
for hit in response["hits"]["hits"]:
    print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
    print(hit["_source"]["short_question"])
    print(hit["_source"]["answer"])
    print()

id: bH2OMoABX4M0dKrGwsk1, score: 1.9935963
Miért fáj a lábfejem futás közben
Nekem volt a lábfejemben szalaghúzódásom, de az álláskor is fájt, bár ha ültem, akkor bármit tehettem vele, nem éreztem semmit.

id: SH2PMoABX4M0dKrGRdnx, score: 1.9854285
Miért akar a kutyám inkább az óljában aludni és nem a fűtött szobában
Kellő mennyiségű aljszőrzetet növesztett és nem fázik kint. Nincs mit tenned, ha ő jól érzi kint magát, akkor nyugodtan hagyjátok. legyen meg a lehetősége, hogy bemenjen, ha akar, de ha nem akar, akkor békén kell hagyni.

id: U36SMoABX4M0dKrG9ksN, score: 1.9847063
Maszturbálás közben miért fáj a fejem 20f
Rossz keringésre utal. Nekem csak nagyon másnaposan szokott lenni. (ami nyilván megmagyarázza)

id: n36VMoABX4M0dKrGwaFK, score: 1.9834182
Menstruáció első két napján furcsa fájdalomszerű érzés lábakban mitől
Nekem is minden alkalommal ilyen szokott lenni.Olyan érzés, mintha nem.jutna oda elég vér.Igen, azt mondta a nődoki is, keringési probléma, és lassabban termeli viss